# Text classification of tweets:

In [1]:
from __future__ import print_function
import numpy as np
import gensim
from gensim.models import Word2Vec, Doc2Vec, FastText
import gensim.parsing.preprocessing as prep
import csv
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import datasets, svm
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
#import seaborn
#seaborn.use()
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

C:\Users\Magnu\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


starting


In [2]:
def open_file(fileName):
    with open(str(fileName), "r", encoding="utf8") as sample:
        s = sample.readlines()
    return s


def create_X(list_of_tweets, w2v, features):

    # this function needs some love!

    X = np.zeros((len(list_of_tweets), features))

    for indeks, tweet in enumerate(list_of_tweets):
        for word in tweet:
            try:
                X[indeks, :] = X[indeks, :] + model_tot.wv[str(word)]
            except:
                pass
        N = len(tweet)
        if N > 0:
            X[indeks] = X[indeks] / N
    return X


def createWordEmbedding(list_of_tweets, features, epoc, sg=0):
    print(sg)
    model = Word2Vec(
        list_of_tweets, size=features, window=5, min_count=1, workers=4, sg=sg)
    print('word embedding created, start training:')
    model.train(
        list_of_tweets, total_examples=len(list_of_tweets), epochs=epoc)
    return model


def createSentEmbedding(list_of_tweets, features, epoc):
    model = Doc2Vec(list_of_tweets, size=features, min_count=1)
    model.train(
        list_of_tweets, total_examples=len(list_of_tweets), epochs=epoc)
    return model


def save_csv(fileName, test_y):
    ids = np.arange(len(test_y))
    with open(fileName, 'w') as csvfile:
        tempwriter = csv.writer(csvfile)
        tempwriter.writerow(["Id", "Prediction"])
        count = 0
        for row in test_y:
            if row == 0:
                row = -1
            tempwriter.writerow([(ids[count]) + 1, str(row)])
            count = count + 1


def train(method, x, y, x_test):
    met = method.fit(x, y)
    test_y = met.predict(x_test)
    return test_y


def shuffle_tweets(listTweets, y):
    c = list(zip(listTweets, y))
    random.shuffle(c)
    listTweets, y = zip(*c)
    return listTweets, y


def processTrainingData(list_of_tweets):
    list_of_tweets = [prep.strip_short(line) for line in list_of_tweets]
    list_of_tweets = prep.preprocess_documents(list_of_tweets)
    return list_of_tweets


def easyProcess(list_of_tweets):
    list_of_tweets = [prep.strip_short(line) for line in list_of_tweets]
    return list_of_tweets

all functions ready


In [3]:
positive_tweets = open_file("train_pos.txt")
negative_tweets = open_file("train_neg.txt")

positive_tweets = processTrainingData(positive_tweets)
negative_tweets = processTrainingData(negative_tweets)
y = [1] * len(positive_tweets) + [0] * len(negative_tweets)

all_tweets = positive_tweets + negative_tweets
all_tweets, y = shuffle_tweets(all_tweets, y)

200000
loaded and processed short training set


Create word embedding based on full set:

In [5]:
'''
#Creates word embeddings for full training set:

positive_tweets_full = open_file("train_pos_full.txt")
negative_tweets_full = open_file("train_neg_full.txt")
print('all loaded')
positive_tweets_full = processTrainingData(positive_tweets_full)
negative_tweets_full = processTrainingData(negative_tweets_full)
y_full = [1]*len(positive_tweets_full)+[0]*len(negative_tweets_full)
print('finished with processing')
all_tweets_full = positive_tweets_full+negative_tweets_full #list of tweets
all_tweets_full, y_full = shuffle_tweets(all_tweets_full, y_full)
'''

'\n#Creates word embeddings for full training set:\n\npositive_tweets_full = open_file("train_pos_full.txt")\nnegative_tweets_full = open_file("train_neg_full.txt")\nprint(\'all loaded\')\npositive_tweets_full = processTrainingData(positive_tweets_full)\nnegative_tweets_full = processTrainingData(negative_tweets_full)\ny_full = [1]*len(positive_tweets_full)+[0]*len(negative_tweets_full)\nprint(\'finished with processing\')\nall_tweets_full = positive_tweets_full+negative_tweets_full #list of tweets\nall_tweets_full, y_full = shuffle_tweets(all_tweets_full, y_full)\n'

In [6]:
features = 200
epoch = 10
#print('let the word embedding commence')
#model_tot_sg1 = createWordEmbedding(all_tweets_full, features, epoch, sg=1) #word embedding
#model_tot_sg0 = createWordEmbedding(all_tweets_full, features, epoch) #word embedding
#print('word embedding created')
'''
features = 350 # we should find optimal value for this:
epoch = 10 # we should fin optimal value for this:
model_tot = createWordEmbedding(all_tweets, features, epoch) #word embedding
print('word embedding created')
'''

"\nfeatures = 350 # we should find optimal value for this:\nepoch = 10 # we should fin optimal value for this:\nmodel_tot = createWordEmbedding(all_tweets, features, epoch) #word embedding\nprint('word embedding created')\n"

In [7]:
#model_tot_sg1.save('model_tot_sg1_200.word2vec')
#model_tot_sg0.save('model_tot_200.word2vec')

In [8]:
model_tot = Word2Vec.load("model_tot_200.word2vec")
model_tot_sg1 = Word2Vec.load('model_tot_sg1_200.word2vec')

In [9]:
X_old = create_X(all_tweets, model_tot, features)
X_old = preprocessing.scale(X_old)
print('processed feature matrix for tweets')

processed feature matrix for tweets


In [10]:
#b = np.ones((X.shape[0], X.shape[1]+1)); b[:,:-1] = X
X = np.ones((X_old.shape[0], X_old.shape[1] + 1))
X[:, 1:] = X_old

In [11]:
testd_tweets = open_file("test_data.txt")
testd = processTrainingData(testd_tweets)
print('loaded test file')
print(testd_tweets[:2])
print(testd[:2])

loaded test file
['1,sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url>\n', "2,<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !\n"]


In [12]:
X_test_old = create_X(testd, model_tot, features)
X_test_old = preprocessing.scale(X_test_old)
X_test = np.ones((X_test_old.shape[0], X_test_old.shape[1] + 1))
X_test[:, 1:] = X_test_old
print('start training')

# Build logistic regression classifiers to identify the polarity of words
#test_y_lr = train(LogisticRegression(solver='lbfgs', max_iter = 1000), X, y, X_test)

# Build naive bayes classifiers to identify the polarity of words
#test_y_nb = train(nb.GaussianNB(), X, y, X_test) # this one isn't working

#print('created classification for submission')

#save_csv('test_resultLR.csv', test_y_lr)
#print('saved')

start training


## Doing some cross validation:

In [13]:
cv_results_lr = cross_validate(
    LogisticRegression(solver='lbfgs'), X, y, return_train_score=False, cv=5)
print(np.mean(np.asarray(cv_results_lr['test_score'])))

0.768815
done with cross validation


In [14]:
positive = open_file("train_pos.txt")
negative = open_file("train_neg.txt")

list_all_tweets = positive + negative
y_full = [1] * len(positive) + [0] * len(negative)
print(len(y))
list_all_tweets, y_full = shuffle_tweets(list_all_tweets, y_full)
list_all_tweets = easyProcess(list_all_tweets)
print('shuffled')

200000
shuffled


In [15]:
SEED = 42
x_train, x_validation, y_train, y_validation = train_test_split(
    list_all_tweets, y_full, test_size=.1, random_state=SEED)

In [16]:
print(len(x_train), len(y_train))

180000 180000


In [17]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [18]:
from gensim.models import KeyedVectors
model_tot_k = KeyedVectors.load('model_tot_200.word2vec')
model_tot_k2 = KeyedVectors.load('model_tot_sg1_200.word2vec')

In [19]:
len(model_tot_k.wv.vocab.keys())

375351

In [20]:
embeddings_index = {}
for w in model_tot_k.wv.vocab.keys():
    embeddings_index[w] = model_tot_k.wv[
        w]  #np.append(model_tot_k.wv[w],model_tot_k2.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 375351 word vectors.


In [21]:
for x in x_train[:5]:
    print(x)

hubby #toosexy bday ... had lots fun #umph <url>
justice vol hardcover eisner award-winning painter alex ross has stunned fans time and again with his ... <url>
hmm that's cute watch being passed around des moines and posted facebook
dont forget please ...
<user> because shes dog and very playfull lol


In [22]:
print(len(x_train))

180000


In [23]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [25]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)
length = max(length) + 2

In [26]:
x_train_seq = pad_sequences(sequences, maxlen=length)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (180000, 35)


In [27]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=length)

In [28]:
print(features)

200


In [34]:
num_words = 100000
embedding_matrix = np.zeros((num_words, features))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [36]:
print(len(x_train), len(y_train), len(x_train_seq))

180000 180000 180000


In [38]:
hFeatures = features * 0.5
structure_test = Sequential()
e = Embedding(100000, features, input_length=length)
structure_test.add(e)
structure_test.add(
    Conv1D(
        filters=100,
        kernel_size=2,
        padding='valid',
        activation='relu',
        strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 200)           20000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 34, 100)           40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


In [41]:
model_cnn_03 = Sequential()
e = Embedding(
    100000,
    features,
    weights=[embedding_matrix],
    input_length=length,
    trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(
    Conv1D(
        filters=32,
        kernel_size=2,
        padding='valid',
        activation='relu',
        strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(128, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(
    x_train_seq,
    y_train,
    validation_data=(x_val_seq, y_validation),
    epochs=2,
    batch_size=32,
    verbose=2)

Train on 180000 samples, validate on 20000 samples
Epoch 1/3


KeyboardInterrupt: 

In [ ]:
tweet_input = Input(shape=(length,), dtype='int32')

tweet_encoder = Embedding(100000, features, weights=[embedding_matrix], input_length=length, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=features*0.5, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=features*0.5, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=features*0.5, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [ ]:
filepath = "CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(
    x_train_seq,
    y_train,
    batch_size=32,
    epochs=2,
    validation_data=(x_val_seq, y_validation),
    callbacks=[checkpoint])

In [ ]:

loaded_CNN_model = load_model('CNN_best_weights.02-0.8333.hdf5')
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

In [ ]:
testd_tweets = easyProcess(testd_tweets)

sequences_test = tokenizer.texts_to_sequences(testd_tweets)
x_test_seq = pad_sequences(sequences_test, maxlen=length)
ycnn01 = loaded_CNN_model.predict_classes(x=x_test_seq)
ycnn02 = loaded_CNN_model.predict(x=x_test_seq)

In [ ]:
positive = open_file("train_pos_full.txt")
negative = open_file("train_neg_full.txt")

list_all_tweets = positive + negative
y_full = [1] * len(positive) + [0] * len(negative)
print(len(y))
list_all_tweets, y_full = shuffle_tweets(list_all_tweets, y_full)
print('shuffled')

In [ ]:
tvec = TfidfVectorizer(max_features=100000, ngram_range=(1, 3))
print('start fitting')
tvec.fit(list_all_tweets)
print('done fitting')

In [ ]:
x_train_tfidf = tvec.transform(list_all_tweets)
x_test_tfidf = tvec.transform(testd_tweets)
print('start LR')
lr_with_tfidf = LogisticRegression(solver='lbfgs', max_iter=1000)
lr_with_tfidf.fit(x_train_tfidf, y_full)
print('Predict LR')
testy = lr_with_tfidf.predict(x_test_tfidf)
save_csv('test_resulttfid.csv', testy)
print('saved')

In [ ]:
cv_results_lr = cross_validate(
    LogisticRegression(solver='lbfgs', max_iter=500),
    x_train_tfidf,
    y_full,
    return_train_score=False,
    cv=5)
#print(cv_results_lr['test_score'])
print(np.mean(np.asarray(cv_results_lr['test_score'])))